# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [2]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 7.4 MB/s eta 0:00:00


In [3]:
import redis

In [4]:
r = redis.Redis(host='redis', port=6379)

In [5]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [6]:
r.set('greetings' , 'hello world')
value = r.get('greetings')
print(value)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [9]:
r.set('connection', '0')
n_value = r.incr('connection')
print(n_value)

1


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [10]:
r.rpush("colors", 'red', 'black','blue')

3

**Q** : Display the second element of the list with `LINDEX`

In [13]:
print(r.lindex("colors", 1))

b'black'


**Q** : Display the full list with `LRANGE`

In [17]:
print(r.lrange("colors", 0, 2))

[b'red', b'black', b'blue']


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [18]:
nosql = ['redis', 'mongodb', 'orientdb']
r.sadd('nosql', *nosql)

3

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [21]:
r.sismember('nosql', 'mysql')

0

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [24]:
r.sadd('nosql', 'hbase')
value = r.smembers('nosql')
print(value)
r.sadd('nosql', 'redis')

{b'mongodb', b'orientdb', b'redis', b'hbase'}


0

### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [25]:
r.zadd('top14', {'Agen' : 10, 'Bordeaux' : 33, 'Brive' : 32, 'Castres' : 29, 'Clermont' : 38, 'La Rochelle' : 26, 'Montpellier' : 32, 'Oyonnax' : 14, 'Pau' : 14, 'Racing' : 40, 'Stade Français' : 22, 'Toulon' : 36, 'Toulouse' : 36}) 

13

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [27]:
score = r.zscore('top14', 'Toulon')
print(score)
pos = r.zrank('top14', 'Toulon')
print(pos)

36.0
9


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [28]:
r.zrevrank('top14', 'Toulon')

3

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [31]:
top3 = r.zrevrange('top14', 0, 3 - 1, withscores=True)
print(top3)
ov35 = r.zrangebyscore('top14', 35, '+inf')
ov35

[(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]


[b'Toulon', b'Toulouse', b'Clermont', b'Racing']

### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [32]:
r.hmset('user:1', {'id' : 1, 'name' : 'Jean', 'age' : 22})
r.hgetall('user:1')

/tmp/ipykernel_114/2562570764.py:1: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset('user:1', {'id' : 1, 'name' : 'Jean', 'age' : 22})


{b'id': b'1', b'name': b'Jean', b'age': b'22'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [45]:
r.hset('user:1', 'city', 'Lyon')
r.hset('user:1', 'name', 'Paul')
r.hgetall('user:1')

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [49]:
import json

http_method = 'PUT'
url = 'my-api.fr/user/10'  # The URL without the protocol

put_request_content = {"id": 10, "name": "Jean"}

json_data = json.dumps(put_request_content)

r.hset(http_method, url, json_data)

1

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [50]:

# HTTP method and URL for the GET request
http_method = 'GET'
url = 'my-api.fr/user?size=2'  # The URL without the protocol

# Content of the GET request
get_request_content = [
    {"id": 10, "name": "Jean"},
    {"id": 11, "name": "Claire"}
]

# Convert the content to a JSON string
json_data = json.dumps(get_request_content)

# Use HSET to store the request information in a Redis hash
r.hset(http_method, url, json_data)

1

**Q** : Create a set of cache request keys.

In [52]:
cache_request_keys = [
    ('GET', 'my-api.fr/user?size=2'),
    ('PUT', 'my-api.fr/user/10'),
    # Add more keys as needed
]

cache_request_set = 'cache_request_keys'

for method, url in cache_request_keys:
    r.sadd(cache_request_set, f'{method} {url}')


**Q** : Check if the GET request on http: //http://my-api.fr/user

In [51]:
stored_data = r.hget(http_method, url)
print(f"Data stored for {http_method} {url}: {stored_data}")

Data stored for GET my-api.fr/user?size=2: b'[{"id": 10, "name": "Jean"}, {"id": 11, "name": "Claire"}]'


**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [55]:
http_method = 'PUT'
url = 'my-api.fr/user/10'

cache_request_set = 'cache_request_keys'

r.hdel(http_method, url)

r.srem(cache_request_set, f'{http_method} {url}')

cached_keys = r.smembers(cache_request_set)
print(f"Updated Cache request keys: {cached_keys}")

Updated Cache request keys: {b'GET my-api.fr/user?size=2'}


## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [56]:
!redis-cli flushall

/bin/bash: line 1: redis-cli: command not found
